In [ ]:
from scCRT.utils_func import *
from scCRT.Estimate import Estimate_time
device = torch.device('cpu')


import umap
import umap.plot

import matplotlib.pyplot as plt
import matplotlib

In [ ]:
############################## 1. Given the raw count and preprocess data  ##############################
'''
actual topology of binary_tree_8
M1 --> M3
M3 --> M2
M3 --> M7
M7 --> M5
M5 --> M4
M5 --> M6
M7 --> M8
'''
# from .rds
# dataset_path = 'src/scCRT/data/binary_tree_8.rds'
dataset_path = '/Users/seutatsu/PycharmProjects/scCRT/src/scCRT/data/binary_tree_8.rds'
dataset_label_path = None

# from .csv
# dataset_path = 'src/scCRT/data/binary_tree_8.csv'
# dataset_label_path = 'src/scCRT/data/binary_tree_8_label.csv'



## get and preprocess data
# get_data
data_counts, cell_labels, cells, name2ids, ids2name, cell_times, pre_infos = getInputData(dataset_path, 
                                                                                          dataset_label_path)

# preprocess
if cell_labels is None:
    features, cell_labels = pre_process(data_counts, WithoutLabel=True)
else:
    features = pre_process(data_counts)

In [ ]:
############################## 2. learn cell features  ##############################
'''
Parameters
----------
input: 
features: the preprocessed expression matrix, [n_cell, n_genes], numpy.array
cell_labels: the label of cells, numpy.array

output: 
y_features: the learned cell features, [n_cell, feature_size]
'''

model_path = 'scCRT/data/binary_tree_8_model.pkl'
y_features = trainingModel(features, cell_labels, device, hidden=[128, 16],  k=20, epochs=200, model_path=model_path)

In [ ]:
############################## 3. get trajectory ##############################
# given the start node
if pre_infos is not None:
    start_node = name2ids[str(pandas2ri.rpy2py(pre_infos[2])[0])]
else:
    start_node=0

# get the clusters
trajectory, network, evaluation_details = get_trajectory(cell_labels, 
            y_features, ids2name, cells, start_node=start_node, norm=False, k=20)

for [s,t] in zip(network[0], network[1]):
    print(f'{s} --> {t}')

In [ ]:
############################## 3.1 evaluation the trajectory ##############################
# calculate HIM, F1-branches, F1-milestones using dyneval (need to install dyneval of R version)
if '.rds' in dataset_path:
    HIM, F1_branches, F1_milestones = evaluation(pre_infos, evaluation_details)
    print('HIM:{:.3f}, F1-branches:{:.3f}, F1-milestones:{:.3f}'.format(HIM, F1_branches, F1_milestones))

In [ ]:
############################## 4. estimate pseudotimes ##############################
time_model = Estimate_time(y_features, cell_labels, start_node=start_node, 
                          k=20)
time_model.fit(num_epochs=1)

predict_times = time_model.unified_pseudotime.astype(float)

In [ ]:
# (1) the PCCs of all cells
total_time = np.corrcoef(predict_times, cell_times.astype(float))[1,0]

# (2) the average PCCs of each lineage
avg_times = []
for one_line, curve in zip(time_model.Lineage_class.lineages, time_model.curves):
    cell_mask = np.logical_or.reduce(np.array([cell_labels == k for k in one_line.lineage]))
    pseudotime = curve.pseudotimes_interp[cell_mask] # 预测顺序
    select_cells_times = cell_times[cell_mask]
    avg_times.append(np.corrcoef(pseudotime.astype(float), select_cells_times.astype(float))[1,0])

avg_time = np.mean(np.array(avg_times))

In [ ]:
avg_time